## Documentation

- API Guide can be found in this link https://ib-insync.readthedocs.io/api.html
- Options Example (SPX) with ib_insync https://nbviewer.org/github/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb


In [15]:
from ib_insync import *
import pandas as pd
util.startLoop()
from datetime import date
from datetime import datetime

def verify_daycount(dates):
    ds = []
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    for adate in dates:
        date_obj = datetime.strptime(adate, "%Y%m%d")
        days_diff = (date_obj - today).days
        if (40 < days_diff < 50) or (90 < days_diff < 150):
            ds.append(adate)
    return ds
    

ib = IB()
ib.connect('127.0.0.1', 7496, clientId = 4)

tick = 'AAPL'
print( " >>> DEFINING STOCK ")
stk = Stock(tick, 'SMART', 'USD')
ib.qualifyContracts(stk)
ib.reqMarketDataType(4) #wait a bit
chains = ib.reqSecDefOptParams(stk.symbol, '', stk.secType, stk.conId)
ds_chains = pd.DataFrame(chains)
expirations = sorted(ds_chains['expirations'].iloc[0])
expirations = verify_daycount(expirations) #convert expiration day to number of days left to expire
strikes = ds_chains['strikes'].iloc[0]
contractID = ds_chains['underlyingConId'].iloc[0]
rights = ['P', 'C'] #easier for algirthm built # P = Put ; C = Call

#organize all Option Contracts into data set
print( " >>> DEFINING OPTIONS ")
contracts = [Option(tick, ex, s, r, 'SMART') for r in rights for ex in expirations for s in strikes]
print( " >>> QUALIFYING OPTIONS ")
qualifiedContracts = ib.qualifyContracts(*contracts)
print( " >>> RETRIEVING DETAILS ")
qualifiedContractDetails = ib.reqTickers(*qualifiedContracts)
ds = pd.DataFrame()
print ( " >>> GETTING GREEKS AND OPTIONS INFO")
for qcd in qualifiedContractDetails:
    try:
        details = qcd.contract
        exp, strike, right = details.lastTradeDateOrContractMonth, details.strike, details.right
        greek = qcd.modelGreeks
        delta, theta, gamma, vega, pvDiv, optPrice, impliedVol = greek.delta, greek.theta, greek.gamma, greek.vega, greek.pvDividend, greek.optPrice, greek.impliedVol
        row = {
            'Symbol': tick, 'Expiration': exp , 'Strike': strike, 'Right': right,
            'Delta': delta, 'Theta': theta, 'Vega': vega, 'Gamma': gamma,
            'impVol': impliedVol, 'optPrice': optPrice, 'Ask': qcd.ask, 'Bid': qcd.bid, 'Last': qcd.last}
        ds = pd.concat([ds, pd.DataFrame([row])], axis = 0, ignore_index = True)
    except:
        print ( " >>> NOGOOD CONTRACT")

#next: call all options during market time to get greek values
def days_diff(adate):
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    date_obj = datetime.strptime(adate, "%Y%m%d")
    return (date_obj - today).days

ds['daysdiff'] = list(map(days_diff, ds['Expiration'])) 
ds['Delta'] = abs(ds['Delta'])
ds['Theta'] = abs(ds['Theta'])
print(ds)

# ds_p1 = ds[(0.1 <= ds['Delta'] <= 0.2) & (ds['Right'] == 'P') & (days_diff(ds['Expiration'], 90, 150))]
ds_p1 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= 90) & (ds['daysdiff'] <= 150) & (ds['Delta'] >= 0.1) & (ds['Delta'] <= 0.2)]
ds_p1['REGEL'] = ['P1'] * len(ds_p1)
ds_p2 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= 40) & (ds['daysdiff'] <= 50) & (ds['Delta'] >= 0.2) & (ds['Delta'] <= 0.3)]
ds_p2['REGEL'] = ['P2'] * len(ds_p2)
ds_c3 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= 40) & (ds['daysdiff'] <= 50) & (ds['Delta'] >= 0.2) & (ds['Delta'] <= 0.3)]
ds_c3['REGEL'] = ['C3'] * len(ds_c3)
ds_c4 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= 90) & (ds['daysdiff'] <= 150) & (ds['Delta'] >= 0.1) & (ds['Delta'] <= 0.2)]
ds_c4['REGEL'] = ['C4'] * len(ds_c4)

main = pd.concat([ds_p1, ds_p2, ds_c3, ds_c4], ignore_index=True)
def p_marg(p1, p2):
    if (p1 == p2 - 10) | (p1 == p2 - 20):
        return True
    return False

def c_marg(c3, c4):
    if (c4 == c3 + 10) | (c4 == c3 + 20):
        return True
    return False

def p_theta(p1, p2):
    if (p2 > p1 * 1.5):
        return True
    return False

def c_theta(c3, c4):
    if (c3 > c4 * 1.5):
        return True
    return False

def bundle(ds):
    p1 = ds[ds['REGEL'] == 'P1']
    p2 = ds[ds['REGEL'] == 'P2']
    c3 = ds[ds['REGEL'] == 'C3']
    c4 = ds[ds['REGEL'] == 'C4']
    ps = []
    cs = []
    i = 0

    #convert two loops to function prperly (rearrange functions in sheet)
    for i in range(len(p1)):
        j = 0
        for j in range(len(p2)):
            # code begin
            if p_marg(p1.iloc[i]['Strike'], p2.iloc[j]['Strike']) & p_theta(p1.iloc[i]['Theta'], p2.iloc[j]['Theta']):
                ps.append((i, j))
            # code end
            j += 1
        i += 1

    for i in range(len(c3)):
        j = 0
        for j in range(len(c4)):
            # code begin
            if c_marg(c3.iloc[i]['Strike'], c4.iloc[j]['Strike']) & c_theta(c3.iloc[i]['Theta'], c4.iloc[j]['Theta']):
                cs.append((i, j))
            # code end
            j += 1
        i += 1
    

    legs_indx = [(p1, p2, c3, c4) for (p1, p2) in ps for (c3, c4) in cs]

    dss = []
    for (a, b, c, d) in legs_indx:
        ds = pd.concat([p1.iloc[a].to_frame().T, p2.iloc[b].to_frame().T, c3.iloc[c].to_frame().T, c4.iloc[d].to_frame().T], ignore_index=True)
        # df = pd.DataFrame.from_records(ds)
        dss.append(ds)
    
    return dss


bundles = bundle(main)
pd.concat(bundles).to_csv('options_03.csv')
ib.disconnect()


 >>> DEFINING STOCK 


Error 200, reqId 5: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221021', strike=30.0, right='P', exchange='SMART')
Error 200, reqId 6: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221021', strike=35.0, right='P', exchange='SMART')
Error 200, reqId 8: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221021', strike=41.25, right='P', exchange='SMART')
Error 200, reqId 7: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221021', strike=40.0, right='P', exchange='SMART')
Error 200, reqId 9: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221021', strike=42.5, right='P', exchange='SMART')
Error 200, reqId 10: No security definition has b

 >>> DEFINING OPTIONS 
 >>> QUALIFYING OPTIONS 


Error 200, reqId 35: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221021', strike=87.5, right='P', exchange='SMART')
Error 200, reqId 39: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221021', strike=97.5, right='P', exchange='SMART')
Error 200, reqId 41: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221021', strike=102.5, right='P', exchange='SMART')
Error 200, reqId 43: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221021', strike=107.5, right='P', exchange='SMART')
Error 200, reqId 45: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221021', strike=112.5, right='P', exchange='SMART')
Error 200, reqId 47: No security definitio

 >>> RETRIEVING DETAILS 
 >>> GETTING GREEKS AND OPTIONS INFO
    Symbol Expiration  Strike Right     Delta     Theta      Vega     Gamma  \
0     AAPL   20221021    50.0     P  0.001521  0.001520  0.004101  0.000083   
1     AAPL   20221021    60.0     P  0.002529  0.002124  0.007659  0.000151   
2     AAPL   20221021    70.0     P  0.004907  0.003461  0.013522  0.000303   
3     AAPL   20221021    75.0     P  0.007056  0.004537  0.014022  0.000438   
4     AAPL   20221021    80.0     P  0.010085  0.005943  0.022677  0.000626   
..     ...        ...     ...   ...       ...       ...       ...       ...   
165   AAPL   20221118   260.0     C  0.001576  0.000576  0.004934  0.000187   
166   AAPL   20221118   270.0     C  0.001735  0.000677  0.005953  0.000190   
167   AAPL   20221118   280.0     C  0.001790  0.000740  0.006227  0.000184   
168   AAPL   20221118   290.0     C  0.001658  0.000719  0.004884  0.000164   
169   AAPL   20221118   300.0     C  0.001498  0.000679  0.004137  0.

<ipython-input-15-d71df50d120f>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_p1['REGEL'] = ['P1'] * len(ds_p1)
<ipython-input-15-d71df50d120f>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_c4['REGEL'] = ['C4'] * len(ds_c4)


ValueError: No objects to concatenate

In [14]:
print(bundles)
print(main)
ib.disconnect()

[]
  Symbol Expiration  Strike Right     Delta     Theta      Vega     Gamma  \
0   AAPL   20221021   120.0     P  0.107804  0.029433  0.143686  0.005925   
1   AAPL   20221021   125.0     P  0.141216  0.034084  0.179983  0.007451   
2   AAPL   20221021   130.0     P  0.182898  0.038778  0.216511  0.009175   
3   AAPL   20221118   115.0     P  0.102682  0.025526  0.158222  0.004867   
4   AAPL   20221118   120.0     P  0.130714  0.029196  0.198440  0.005990   
5   AAPL   20221118   125.0     P  0.164631  0.032769  0.205409  0.007254   
6   AAPL   20221021   175.0     C  0.175508  0.030636  0.188670  0.012196   
7   AAPL   20221021   180.0     C  0.124211  0.023761  0.152032  0.009828   
8   AAPL   20221118   180.0     C  0.174408  0.027804  0.219179  0.010385   
9   AAPL   20221118   185.0     C  0.131514  0.022689  0.177050  0.008708   

     impVol  optPrice   Ask   Bid  Last  daysdiff REGEL  
0  0.407280  1.788522  1.80  1.78   NaN        95    P1  
1  0.391918  2.392336  2.40  2.38

In [13]:
import openpyxl
bundles.to_excel('pandas_to_excel.xlsx', sheet_name='new_sheet_name')

writer = pd.ExcelWriter('bundles.xlsx',engine='xlsxwriter')
workbook = writer.book
worksheet = workbook.add_worksheet('Result')
writer.sheets['Bundles'] = worksheet

bundle_count = 1
currow, curcol = 0, 0
for ds in bundles:
    worksheet.write_string(currow, 0, str(ds[ds['REGEL'] == 'P1']['Symbol']) + ": Bundle " + str(bundle_count))
    currow += 1
    ds['Change'] = ((ds['Strike'] - ds['Last'])/ds['Last']) * 100 #convert to percentage
    ds['Mark'] = (ds['Ask'] + ds['Bid']) / 2
    ds['impVol'] = ds['impVol'] * 100
    ds['RTD'] = [0] * len(ds)
    ds['Margin'] = [0] * len(ds)
    ds[(ds['REGEL'] == 'P1') | (ds['REGEL'] == 'P2')]['RTD'] = ds[ds['REGEL'] == 'P2']['Theta'] - ds[ds['REGEL'] == 'P1']['Theta']
    ds[(ds['REGEL'] == 'C3') | (ds['REGEL'] == 'C4')]['RTD'] = ds[ds['REGEL'] == 'C3']['Theta'] - ds[ds['REGEL'] == 'C4']['Theta']
    ds[(ds['REGEL'] == 'P1') | (ds['REGEL'] == 'P2')]['Margin'] = ds[ds['REGEL'] == 'P2']['Strike'] - ds[ds['REGEL'] == 'P1']['Strike']
    ds[(ds['REGEL'] == 'C3') | (ds['REGEL'] == 'C4')]['Margin'] = ds[ds['REGEL'] == 'C4']['Strike'] - ds[ds['REGEL'] == 'C3']['Strike']
    ds['RTM'] = ds['RTD'] * 22
    ds[['REGEL', 'Expiration', 'daysdiff', 'Strike', 'Change', 'Mark', 'Delta', 'Theta', 'Vega', 'impVol', 'RTM']].to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
    currow += ds.shape[0] + 2


    symbol = str(ds[ds['REGEL'] == 'P1']['Symbol'])
    pnative = {
            'Type': symbol, 
            'Margin': ds[ds['REGEL'] == 'P1']['Margin'],
            'Monthly%': ds[ds['REGEL'] == 'P1']['RTM'],
            'Yearly%': (ds[ds['REGEL'] == 'P1']['RTM'] * 12 * 100) / ds[ds['REGEL'] == 'P1']['Margin'],
            'Yearly$': (ds[ds['REGEL'] == 'P1']['RTM'] * 12),
            "Cost": ds[ds['REGEL'] == 'P1']['Mark'] + ds[ds['REGEL'] == 'P2']['Mark']
    }
    cnative = {
            'Type': symbol, 
            'Margin': ds[ds['REGEL'] == 'C3']['Margin'],
            'Monthly%': ds[ds['REGEL'] == 'C3']['RTM'],
            'Yearly%': (ds[ds['REGEL'] == 'C3']['RTM'] * 12 * 100) / ds[ds['REGEL'] == 'C3']['Margin'],
            'Yearly$': (ds[ds['REGEL'] == 'C3']['RTM'] * 12),
            "Cost": ds[ds['REGEL'] == 'C3']['Mark'] + ds[ds['REGEL'] == 'C4']['Mark']
    }
    posnative = {
            'Type': symbol, 
            'Margin': max(pnative.get('Margin'), cnative.get('Margin')),
            'Monthly%': (ds[ds['REGEL'] == 'C3']['RTM'] + ds[ds['REGEL'] == 'P1']['RTM']) / max(pnative.get('Margin'), cnative.get('Margin')),
            'Yearly%': ((ds[ds['REGEL'] == 'P1']['RTM'] * 12) + (ds[ds['REGEL'] == 'C3']['RTM'] * 12)) / max(pnative.get('Margin'), cnative.get('Margin')),
            'Yearly$': (pnative.get('Yearly$') + cnative.get('Yearly$')),
            "Cost": ds['Mark'].sum()
    }
    ds_ = pd.DataFrame([pnative, cnative, posnative])
    ds_.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
    currow += 10 # 4 for the native ds_, 6 for spacing
    bundle_count += 1

writer.save() #save the excelt sheet

IndentationError: expected an indented block (<ipython-input-13-65afc5882df4>, line 12)

In [8]:
# ds = pd.read_csv('options_03.csv')
with open('main_options.csv','a') as f:
    for ds in bundles:
        f.write("\n")
        ds['RTM'] = [0] * len(ds) #CREATE EMPTY RTM column, fill with forumla later
        ds['Margin'] = [0] * len(ds) #CREATE EMPTY RTM column, fill with forumla later
        ds['RTD'] = [0] * len(ds) #CREATE EMPTY RTD column, fill with forumla later
        ds['Change'] = ((ds['Strike'] - ds['Last'])/ds['Last']) * 100 #convert to percentage
        ds['Mark'] = (ds['Ask'] + ds['Bid']) / 2
        ds[(ds['REGEL'] == 'P1') | (ds['REGEL'] == 'P2')]['Margin'] = ds[ds['REGEL'] == 'P2']['Strike'] - ds[ds['REGEL'] == 'P1']['Strike']
        ds[(ds['REGEL'] == 'P1') | (ds['REGEL'] == 'P2')]['RTD'] = ds[ds['REGEL'] == 'P2']['Theta'] - ds[ds['REGEL'] == 'P1']['Theta']
        ds[(ds['REGEL'] == 'C3') | (ds['REGEL'] == 'C4')]['Margin'] = ds[ds['REGEL'] == 'C4']['Strike'] - ds[ds['REGEL'] == 'C3']['Strike']
        ds[(ds['REGEL'] == 'C3') | (ds['REGEL'] == 'C4')]['RTD'] = ds[ds['REGEL'] == 'C3']['Theta'] - ds[ds['REGEL'] == 'C4']['Theta']
        ds['RTM'] = ds['RTD'] * 22
        ds['RTY'] = ds['RTM'] * 12
        ds['NDR%'] = (ds['RTD']/ds['Margin']) * 100
        ds['NMR%'] = (ds['RTM']/ds['Margin']) * 100
        ds['NYR%'] = (ds['RTY']/ds['Margin']) * 100
        ds['NMR%PP'] = ((ds['RTM'].idxmax() + ds['RTM'].idxmin()) / ds['Margin'].idxmax())
        ds['NYR%PP'] = ((ds['RTY'].idxmax() + ds['RTY'].idxmin()) / ds['Margin'].idxmax())
        ds = ds[['REGEL', 'Expiration', 'daysdiff', 'Strike', 'Change', 'Mark', 'Delta', 'Theta', 'Vega', 'impVol', 'RTD', 'RTM', 'RTY', 'Margin', 'NDR%', 'NMR%', 'NYR%', 'NMR%PP', 'NYR%PP']]
        ds.to_csv(f)




In [ ]:
[Ticker(
    contract=Option(symbol='SPY', lastTradeDateOrContractMonth='20180420', strike=270, right='C', exchange='SMART'), 
    time=datetime.datetime(2018, 3, 1, 16, 1, 17, 467516, tzinfo=datetime.timezone.utc), 
    bid=6.83, bidSize=1156, ask=6.88, askSize=11, last=6.88, lastSize=1, volume=2264, high=7.67, low=6.17, close=6.95, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[], 
    bidGreeks=OptionComputation(impliedVol=0.160298280414867, delta=None, optPrice=6.829999923706055, pvDividend=1.0992683616305827, gamma=None, vega=None, theta=None, undPrice=271.32000732421875), 
    askGreeks=OptionComputation(impliedVol=0.16155485178197637, delta=None, optPrice=6.880000114440918, pvDividend=1.0992683616305827, gamma=None, vega=None, theta=None, undPrice=271.32000732421875), 
    lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=6.880000114440918, pvDividend=1.0992683616305827, gamma=None, vega=None, theta=None, undPrice=271.32000732421875), 
    modelGreeks=OptionComputation(impliedVol=0.15770592183303841, delta=0.5344203452034115, optPrice=6.665772690114447, pvDividend=1.0992683616305827, gamma=0.026063908021373355, vega=0.3941542814583521, theta=-0.07095144483744885, undPrice=271.32000732421875)
)]
